# Whitepaper plot

In [0]:
# Number of points within rectangular selection
mask = get_selection_mask(obs, nb=False, probcut=None, observed=True)
print('obs:', mask.shape, mask.sum())

mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)
mask = sim.apply_categories(mask)
print('sim:', mask.shape, mask.sum(axis=0))

In [0]:
w1 = np.array([2.05946072e-04, 8.57202980e-05, 3.65160318e-02, 1.48482228e-02,
 5.69068666e-04, 2.49689236e-04, 5.41036362e-02, 2.27939918e-02,
 1.33515545e-01, 5.77029378e-02, 4.76497084e-01, 2.02912126e-01])

w2 = np.array([2.12000e-04, 7.60000e-05, 3.66160e-02, 1.47720e-02, 5.36000e-04, 2.88000e-04,
 5.34720e-02, 2.30760e-02, 1.34424e-01, 5.76160e-02, 4.77500e-01, 2.01412e-01])

In [0]:
# Population weights without NB cuts
print('w1', w1, w1.sum())

# New categories
g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
print('g1', g1.shape)

# Verify new categories
w2 = np.bincount(g1) / g1.shape
print('w2', w2, w2.sum(), np.sum(w2[:-2]), np.sum(w2[-2:]))

# Population weights after cuts
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)
sim.apply_categories(mask, g=g1)
w3 = np.bincount(g1[mask[:,0]]) / mask.sum()
w3 /= w3.sum()
print('w3', w3, np.sum(w3), np.sum(w3[:-2]), np.sum(w3[-2:]))

# Population weights for the ghost plot
w = np.bincount(sim.data['g']) / sim.data['g'].shape
print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

# New weights, boost thick disk and halo
w4 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
w4[2:4] *= 100
# w4[0:6] *= 3  # thin disk 1-3
# w4[4:6] *= 1.2  # thin disk 3
w4[6:8] *= 15  # thick disk
w4[8:10] *= 28   # halo
w4[10:12] *= 50  # dSph
w4 /= w4.sum()
print('w4', w4, np.sum(w4[:-2]), np.sum(w4[-2:]))

In [0]:
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)

pmap_nonb = ProbabilityMap(cmd.axes)
pmap_nonb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w1, observed=True, mask=mask)
pmap_nonb.maximum_filter()

pmap_samp = ProbabilityMap(cmd.axes)
pmap_samp.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w4, observed=True, mask=mask)
pmap_samp.maximum_filter()

mask = get_selection_mask(sim, nb=True, probcut=None, observed=True)

pmap_nb = ProbabilityMap(cmd.axes)
pmap_nb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w3, observed=True, mask=mask)
pmap_nb.maximum_filter()

In [0]:
psamp = ProbabilitySampling(pmap_samp, 0)
psamp_mask = psamp.apply(obs, observed=True)

In [0]:
def hex_to_rgb(hex):
    return tuple(int(hex[i + 1:i + 3], 16) / 255 for i in (0, 2, 4))

def chop_cmap_frac(cmap: LinearSegmentedColormap, frac: float) -> LinearSegmentedColormap:
    """Chops off the beginning `frac` fraction of a colormap."""
    cmap_as_array = cmap(np.arange(256))
    cmap_as_array = cmap_as_array[:int(frac * len(cmap_as_array))]
    return LinearSegmentedColormap.from_list(cmap.name + f"_frac{frac}", cmap_as_array)

def get_nice_cmap(from_black=False):
    if from_black:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, hex_to_rgb('#595959')),
            (0.33, hex_to_rgb('#5F9ED1')),
            (0.66, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad((1.0, 1.0, 1.0))
    else:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, (1.0, 1.0, 1.0)),
            (0.15, hex_to_rgb('#ABABAB')),
            (0.25, hex_to_rgb('#595959')),
            (0.50, hex_to_rgb('#5F9ED1')),
            (0.75, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad(cmap(0))

    return cmap

#cmap = mpl.cm.get_cmap('terrain_r')

# cmap = LinearSegmentedColormap.from_list('mycmap',
#         [(0.00, (1.0, 1.0, 1.0)),
#          #(0.15, (0.75, 0.75, 0.75)),
#          (0.25, (1.0, 1.0, 0.6)),
#          (0.50, (0.0, 0.8, 0.4)),
#          (0.75, (0.0, 0.6, 1.0)),
#          (1.00, (1.0, 0.0, 0.0))])

#006BA4, #FF800E, #ABABAB, #595959,
#5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF

# cmap = mpl.cm.get_cmap('cubehelix_r')
# cmap = chop_cmap_frac(cmap, 0.5)

cmap = get_nice_cmap()

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap())

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap(from_black=True))

In [0]:
gs = GridSpec(ncols=6, nrows=1, width_ratios=[5, 5, 5, 5, 5, 0.3])
f = plt.figure(constrained_layout=False, figsize=(7.2, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(5) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
#cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

cmd.plot_observation(axs[2], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = cmd.plot_probability_map(axs[3], pmap_nonb, 1, cmap=cmap, rasterized=True)
l1 = cmd.plot_probability_map(axs[4], pmap_nb, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min', cmap=cmap)
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.55, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')

In [0]:
probcmap = plt.get_cmap('Greys')

gs = GridSpec(ncols=7, nrows=1, width_ratios=[5, 5, 5, 5, 0.3, 0.6, 0.3])
f = plt.figure(constrained_layout=False, figsize=(4.5, 1.8), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(4) ])
axc = f.add_subplot(gs[0, -2])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
# cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

cmd.plot_observation(axs[2], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = cmd.plot_probability_map(axs[3], pmap_nonb, 1, cmap=probcmap, rasterized=True)
# l1 = cmd.plot_probability_map(axs[4], pmap_nb, 1, cmap=probcmap, rasterized=True)

for l in (l0,):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min', cmap=probcmap)
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.3, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'$\mathrm{mag}~g - \mathrm{mag}~i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.55, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'$\mathrm{mag}~g$')

axs[0].set_title('observation')
axs[1].set_title('simulation')
axs[2].set_title('"ghost plot"')
axs[3].set_title('probability')

f.subplots_adjust(wspace=0)
# f.tight_layout()

f.savefig('obs_sim_prob_poster.pdf')
f.savefig('obs_sim_prob_poster.eps')

In [0]:
gs = GridSpec(ncols=3, nrows=1, width_ratios=[10, 10, 0.5])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

# cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
# #cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

# cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
# #cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
# #cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

# cmd.plot_observation(axs[0], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = cmd.plot_probability_map(axs[0], pmap_nonb, 1, cmap=cmap, rasterized=True)
l1 = cmd.plot_probability_map(axs[1], pmap_nb, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min')
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

for t in axc.get_yticklabels():
    t.set_horizontalalignment('right')   
    t.set_verticalalignment('center')   
    t.set_x(4.2)
    #t.set_y(-1)

f.tight_layout()

f.savefig('umi_hsc_pmap.pdf')

In [0]:
get_selection_mask(obs)

In [0]:
gs = GridSpec(ncols=2, nrows=1, width_ratios=[10, 10])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])

cmd.plot_observation(axs[0], obs, mask=psamp_mask, c='grey', rasterized=True)

psamp_outline_mask = get_selection_mask(obs, nb=True, probcut=None, observed=True)
cmd.plot_observation(axs[1], obs, mask=~psamp_mask & psamp_outline_mask, c='grey', rasterized=True)

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'ab'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel('HSC $g$')
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()

for ax in axs.flatten()[1:]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

f.tight_layout()

f.savefig('umi_hsc_ghost.pdf')

In [0]:
f, axs = plt.subplots(2, 2, figsize=(3.4, 4), dpi=240)

mask = get_selection_mask(obs, nb=True)

cmd.plot_observation(axs[0, 0], obs, c='grey', observed=True, rasterized=True)
cmd.plot_isochrone(axs[0, 0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c=hex_to_rgb('#006BA4'), lw=1)
cmd.plot_isochrone(axs[0, 0], iso_red, observed=True, error_sigma=[2.0, 2.0], c=hex_to_rgb('#FF800E'), lw=1)

cmd.plot_simulation(axs[0, 1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

ccd.plot_observation(axs[1, 0], obs, c='grey', observed=True, rasterized=True)
# ccd.plot_observation(axs[1, 0], obs, c='red', size=3, mask=mask, observed=True, rasterized=True)

ccd.plot_simulation(axs[1, 1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

for ax, pl in zip(axs.flatten(), 'abcd'):
    ax.text(0.1, 0.9, f'{pl})', fontsize=8, transform=ax.transAxes)
    ax.set_xlabel(r'HSC $g - i$')
    #ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    #ax.set_ylim(15.5, 24.5)
    #ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)

for ax in axs[0, :]:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax in axs[1, :]:
    ax.plot([0.12, 2.0, 2.0,
         1, 0.5, 0.5, 0.12, 0.12], 
        
        [0.5, 0.5, 0.1,
         0.1, -0.025, -0.5, -0.5, 0.5], 'k--', lw=1)

axs[0, 0].set_ylabel(r'HSC $g$')
axs[1, 0].set_ylabel(r'HSC $g - $NB$515$')

for ax in axs[:, 1]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

for ax in axs[0, :]:
    ax.set_xlabel(None)
    ax.set_xticklabels([])

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')